In [175]:
from keras.models import Sequential
import pandas as pd
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop, SGD
import sys, re, glob
import numpy as np
from keras.layers import Input, Dense, Flatten

# Data

In [242]:
def load_data(folder, rng):
    X = []

    for i in range(rng):

        packets = list()
        max_node = 0
        max_time = 0

        for file in glob.glob("{}/output_{}/*.routes".format(folder, i)):

            handle = open(file, 'r')
            data = handle.read()
            handle.close()

            _nodes = re.split('\n\n', data);

            _nodes.pop()
            for node in _nodes:

                _strs = re.findall('(\d{1,3}(?:\.\d{1,3}){3})\s+\d{1,3}(?:\.\d{1,3}){3}\s+\d{1,3}(?:\.\d{1,3}){3}\s+\w+\s+-?\d+\.\d+\s+(\d+)', node)

                strings = list()
                for _str in _strs:
                    strings.append(dict(zip(('Destination', 'Hops'), _str)))

                header = re.findall('Node:\s+(\d+)\s+Time:\s+(\d+)', node)

                max_node = max(max_node, int(header[0][0]))
                max_time = max(max_time, int(header[0][1]))

                for _str in strings:
                    _str['Node'] = int(header[0][0])
                    _str['Time'] = int(header[0][1])
                    packets.append(_str)


        table = pd.DataFrame(packets)
        time_agg = table.groupby(["Node", "Destination"]).agg(['min', 'max', "mean", "median", "prod", "sum", "std", "var"])
        node_agg = time_agg.groupby("Node").agg(['min', 'max', "mean", "median", "prod", "sum", "std", "var"])
        aggregate = node_agg.agg(['min', 'max', "mean", "median", "prod", "sum", "std", "var"])
        X.append(aggregate.values.flatten())

    X = np.array(X)
    X[np.isnan(X) or np.isinf(X)] = 0
    
    return X


In [243]:
malicious = load_data("../data/malicious", 100)
normal = load_data("../data/normal", 100)

# Model

In [105]:
SHAPE = malicious.shape[1]

In [259]:
OPTIMIZER = Adam()

In [273]:
model = Sequential()
model.add(Dense(2048, input_shape=(1000,)))
model.add(LeakyReLU(alpha=0.2))
model.add(BatchNormalization(momentum=0.8))
model.add(Dense(SHAPE, activation='sigmoid'))
generator = model

generator.compile(loss='binary_crossentropy', optimizer=OPTIMIZER)

In [274]:
model = Sequential()
model.add(Dense(10, input_shape=(SHAPE,)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(1, activation='sigmoid'))

discriminator = model

discriminator.compile(loss='binary_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'] )

In [275]:
discriminator.trainable = False
model = Sequential()
model.add(generator)
model.add(discriminator)

stacked = model
stacked.compile(loss='binary_crossentropy', optimizer=OPTIMIZER)

# Train

In [276]:
def train(malicious, normal, epochs=200, batch = 10):
    for cnt in range(epochs):
        
        ## train discriminator
        random_index =  np.random.randint(0, len(malicious) - batch)
        malicious_sample = malicious[random_index : random_index + batch]
        normal_sample = normal[random_index : random_index + batch]
        
        gen_noise = np.random.normal(0, 1, (batch,1000))
        syntetic = generator.predict(gen_noise)
        
        x_combined_batch = np.concatenate((malicious_sample, normal_sample, syntetic))
        y_combined_batch = np.concatenate((np.ones((batch, 1)), np.zeros((batch, 1)), np.zeros((batch, 1))))
        
        x_combined_batch =  np.clip(0.999, 0.001, x_combined_batch)
        
        d_loss = discriminator.train_on_batch(x_combined_batch, y_combined_batch)
    
        # train generator
        noise = np.random.normal(0, 1, (batch,1000))
        y_mislabled = np.ones((batch, 1))
        g_loss = stacked.train_on_batch(noise, y_mislabled)
        print ('epoch: %d, [Discriminator :: d_loss: %f], [ Generator :: loss: %f]' % (cnt, d_loss[0], g_loss))

In [277]:
train(malicious, normal)

/home/nsuprotivniy/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch: 0, [Discriminator :: d_loss: 0.764609], [ Generator :: loss: 0.627681]
epoch: 1, [Discriminator :: d_loss: 0.668069], [ Generator :: loss: 1.009454]
epoch: 2, [Discriminator :: d_loss: 0.589651], [ Generator :: loss: 1.562426]
epoch: 3, [Discriminator :: d_loss: 0.620194], [ Generator :: loss: 2.632890]
epoch: 4, [Discriminator :: d_loss: 0.557758], [ Generator :: loss: 2.602346]
epoch: 5, [Discriminator :: d_loss: 0.534219], [ Generator :: loss: 2.985769]
epoch: 6, [Discriminator :: d_loss: 0.541269], [ Generator :: loss: 3.028757]
epoch: 7, [Discriminator :: d_loss: 0.534560], [ Generator :: loss: 3.883941]
epoch: 8, [Discriminator :: d_loss: 0.505951], [ Generator :: loss: 3.452703]
epoch: 9, [Discriminator :: d_loss: 0.478983], [ Generator :: loss: 4.555884]
epoch: 10, [Discriminator :: d_loss: 0.492241], [ Generator :: loss: 4.560247]
epoch: 11, [Discriminator :: d_loss: 0.499176], [ Generator :: loss: 4.450745]
epoch: 12, [Discriminator :: d_loss: 0.549004], [ Generator ::

epoch: 104, [Discriminator :: d_loss: 0.540137], [ Generator :: loss: 3.892890]
epoch: 105, [Discriminator :: d_loss: 0.530446], [ Generator :: loss: 3.228940]
epoch: 106, [Discriminator :: d_loss: 0.553870], [ Generator :: loss: 3.351205]
epoch: 107, [Discriminator :: d_loss: 0.422579], [ Generator :: loss: 3.794232]
epoch: 108, [Discriminator :: d_loss: 0.465310], [ Generator :: loss: 3.646523]
epoch: 109, [Discriminator :: d_loss: 0.485207], [ Generator :: loss: 4.610663]
epoch: 110, [Discriminator :: d_loss: 0.501155], [ Generator :: loss: 4.021140]
epoch: 111, [Discriminator :: d_loss: 0.461219], [ Generator :: loss: 2.767040]
epoch: 112, [Discriminator :: d_loss: 0.473895], [ Generator :: loss: 2.815725]
epoch: 113, [Discriminator :: d_loss: 0.485136], [ Generator :: loss: 2.360297]
epoch: 114, [Discriminator :: d_loss: 0.433853], [ Generator :: loss: 3.916283]
epoch: 115, [Discriminator :: d_loss: 0.463642], [ Generator :: loss: 2.828188]
epoch: 116, [Discriminator :: d_loss: 0.